## Solving SAT-NAE using (3,1)-QRAC

Suppose we take an easy CNF $(x_1 \lor x_2 \lor \neg x_3) \land (x_1 \lor \neg x_2 \lor x_3)$ (we know it has a solution $[1, 0, 0]$), we can transform this into a graph $G = (V,E)$ with 6 nodes (3 for the variables $x_1$ to $x_3$ and 3 for the variables $\neg x_1$ to $\neg x_3$). Previously, we have shown that it is possible to solve 3SAT-NAE on QAOA, but we will attempt to create a Hamiltonian to solve using QRAO for an arbitrary CNF

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import math
from qiskit import IBMQ, QuantumCircuit, transpile
from qiskit.opflow import X, Y, Z, I
from qiskit.algorithms.optimizers import NFT
from qiskit_optimization.runtime import VQEClient
from qiskit.circuit.library import TwoLocal, EfficientSU2

In [2]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub="ibm-q-utokyo", group="internal", project="hirashi-jst")

backend = provider.get_backend("ibmq_qasm_simulator")

/tmp/ipykernel_39505/2703343067.py:1: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  IBMQ.load_account()
/tmp/ipykernel_39505/2703343067.py:1: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  IBMQ.load_account()


In [3]:
# Create a graph from an input CNF formula
def parse_cnf_formula():
    # 1) Used inputs
    num_variables = int(input("Enter the number of variables: "))
    cnf = input("Enter the CNF of the form above (ex: 0 1 2,0 n1 2): ")
    # 2) Separating the CNF into clauses
    clauses = cnf.replace(",", " ")
    parsed_cnf = clauses.split(" ")
    for i in range(len(parsed_cnf)):
        if parsed_cnf[i].startswith("n"):
            parsed_cnf[i] = str(int(parsed_cnf[i][1:])+num_variables)
    parsed_cnf = list(map(lambda elem: int(elem), parsed_cnf))
    set_of_variables = set(parsed_cnf)

    G = nx.Graph()
    vertices = list(set_of_variables)
    G.add_nodes_from(vertices)
    for i in range(0, len(parsed_cnf), 3):
        G.add_edge(parsed_cnf[i], parsed_cnf[i+1], color='b')
        G.add_edge(parsed_cnf[i], parsed_cnf[i+2], color='b')
        G.add_edge(parsed_cnf[i+1], parsed_cnf[i+2], color='b')

    for i in range(len(parsed_cnf)):
        if parsed_cnf[i]+num_variables in set_of_variables:
            G.add_edge(parsed_cnf[i], parsed_cnf[i]+num_variables, color='r')

    
    return G, parsed_cnf, num_variables

In [4]:
# IF WE WANT TO DISPLAY THE GRAPH
# node_colors = ["lightblue"]*obtained_graph.number_of_nodes()
# edge_colors = [obtained_graph[u][v]['color'] for u,v in obtained_graph.edges()]
# nx.draw_circular(obtained_graph, with_labels=True, node_color=node_colors, edge_color=edge_colors)

### Graph coloring
Now we color this graph to assign neighboring nodes a different color

In [31]:
def color_graph(obtained_graph):
    # If we want visualizatoin
    colors_list = ["lightcoral", "lightgray", "yellow", "lime", "lightblue", "green", 
                   "mediumaquamarine", "magenta", "lightsteelblue", "chocolate", "darkorange", "moccasin"]
    #colors = nx.greedy_color(obtained_graph)
    colors = nx.equitable_color(obtained_graph, math.ceil((obtained_graph.number_of_nodes()/3)))
    # new_node_colors = list(map(lambda elem: colors_list[colors[elem]], obtained_graph.nodes))
    # nx.draw_circular(obtained_graph, with_labels=True, node_color=new_node_colors, edge_color=edge_colors)
    return colors

Now that we got our colored graph, we can transform the colors list into a set of list of vertices $V_{c_i}$ where each set contains vertices of the same color $c_i$. We can do this based off the `colors` variable obtained through equitable graph coloring.

In [32]:
def group_nodes_by_color(obtained_graph, colors):
    num_colors = len(set(colors.values()))
    nodes_grouped_by_color = []
    for i in range(num_colors):
        nodes_grouped_by_color.append([])
    for vertex in obtained_graph.nodes():
        nodes_grouped_by_color[colors[vertex]].append(vertex)
    return nodes_grouped_by_color

We need $\lceil\frac{|V_c|}{3}\rceil$ qubits for color $c$. So the total number of qubits needed is going to be $\sum\limits_{c \in C}\lceil\frac{|V_c|}{3}\rceil$. In this next cell, we create the qubit mappings, i.e. which qubit corresponds to which Vertex. The way we proceed is greedily assigning qubits to vertices. (https://arxiv.org/pdf/2302.00429.pdf [paper by Kosei Teramoto])

In [33]:
def create_qv_mapping(nodes_grouped_by_color, bits_per_qubit=3):
    num_qubits_needed = 0
    for i in range(len(nodes_grouped_by_color)):
        num_qubits_needed += math.ceil(len(nodes_grouped_by_color[i])/bits_per_qubit)

    ## qubit q_i maps to a list of vertices associated with qubit q_i (max 3 vertices per qubit)
    qv_mappings = []    

    for elem in nodes_grouped_by_color:
        # this case covers when we need one qubit for a color
        if len(elem) <= bits_per_qubit:
            qv_mappings.append(elem)
        # this case covers when we need more than one qubit for a color (split the list and assign more than one qubits)
        else:
            first_index = 0
            second_index = bits_per_qubit # slicing the list (i.e. second index will be exluded from the slice)
            while first_index < len(elem):
                # make sure not to go too far in the slice in case the length is not a multiple of 3
                if second_index > len(elem):
                    second_index = len(elem)
                    # append sliced list (max size = bits_per_qubit), i.e. if bpq=3 then for [1,2,3,4,5], we append [1,2,3] and [4,5]
                qv_mappings.append(elem[first_index:second_index])
                # increase the indices for appending the next element
                first_index += bits_per_qubit
                second_index += bits_per_qubit
    return qv_mappings

### Now create the variable->qubit mappings

In [34]:
def create_vq_mapping(qv_mapping):
    vq_mapping = {}
    for i in range(len(qv_mapping)):
        for j in range(len(qv_mapping[i])):
            vq_mapping[qv_mapping[i][j]] = i
    return vq_mapping

### Assign X, Y and Z Pauli operators respectively to each vertex per qubit
We will have a "vertex (index) -> Pauli Operator" mapping, to later construct the Hamiltonian based on this mapping

In [35]:
# Maps each vertex to an operator based on the qubit mappings
def create_vp_mapping(qv_mapping):
    mappings = {}

    qrac_3_operators = [X, Y, Z] # Not easy to generalize but we use for (3,1)-QRAC here

    for vertex_groups in qv_mapping:
        for i in range(len(vertex_groups)):
            mappings[vertex_groups[i]] = qrac_3_operators[i] # assign a different operator per vertex in the same group

    return mappings

### Display the operator assignments for each vertex

### Create the Hamiltonian based on the operator assignment for each qubit
We want to find the ground state of $\begin{equation}\sum\limits_{c\in C}(P_iP_j + P_iP_k + P_jP_k) + H_{pen}\end{equation}$.
where $C$ corresponds to the set of clauses of the CNF, $H_{pen}$ is the penalty Hamiltonian of the 3SAT-NAE problem, and $P_l$ is the Pauli operator of vertex $l$ applied on the qubit corresponding to vertex $l$.

To get $H_{pen}$, we can simply replace $Z_iZ_j$ from the original (non-QRAC) Hamiltonian form with $P_iP_j$. Thus our final Hamiltonian for will be given by:
$\begin{equation}\sum\limits_{c\in C}(P_iP_j + P_iP_k + P_jP_k) + M\sum\limits_{v_i \in V(G)} P_{v_i}P_{\neg v_i})\end{equation}$

We will now design this Hamiltonian given an input CNF.

In [36]:
# Create the accumulator Hamiltonian
def create_zero_hamiltonian(num_qubits):
    zero_H = I - I
    for _ in range(num_qubits-1):
        zero_H = zero_H ^ I
    return zero_H

In [37]:
# Create the problem Hamiltonian based on the vertex-to-qubit and vertex-to-operator mappings
def create_problem_hamiltonian(num_qubits, parsed_cnf, vq_mapping, vp_mapping):
    prob_H = create_zero_hamiltonian(num_qubits)
    
    # From the parsed cnf, create (v_i, P_i, Q_i) mappings
    v_p_q = list(map(lambda elem: (elem, vp_mapping[elem], vq_mapping[elem]), parsed_cnf))
    
    # Create groupings by 3 (for the clauses)
    vpq_grouped_by_clause = [v_p_q[i:i + 3] for i in range(0, len(v_p_q), 3)]
    
    for vpq_clause in vpq_grouped_by_clause:
        # Sort by qubit index (because we want to apply operators in order of qubits to the Hamiltonian)
        vpq_clause_sorted = sorted(vpq_clause, key=lambda x: x[2])
        op0 = (vpq_clause_sorted[0], vpq_clause_sorted[1]) # First pair in clause vpq_clause
        op1 = (vpq_clause_sorted[0], vpq_clause_sorted[2]) # Second pair in clause vpq_clause
        op2 = (vpq_clause_sorted[1], vpq_clause_sorted[2]) # Third pair in clause vpq_clause
        op_pairs = [op0, op1, op2]
        for op_pair in op_pairs:
            op_pair_H = 1
            for i in range(num_qubits):
                # If i == index of the qubit we want to apply P on
                if i == op_pair[0][2]:
                    op_pair_H = op_pair_H ^ op_pair[0][1]
                elif i == op_pair[1][2]:
                    op_pair_H = op_pair_H ^ op_pair[1][1]
                else: # Apply I if qubit i does not correspond to any vertex in the pair
                    op_pair_H = op_pair_H ^ I
            prob_H += op_pair_H
    return prob_H

In [38]:
def create_penalty_hamiltonian(num_qubits, parsed_cnf, num_variables, vq_mapping, vp_mapping):
    
    # We will store vertices for which negations exist (we only want to add penalty for the vertices that have negations)
    vertices_with_negation = []
    non_negated_variables = list(range(num_variables))
    set_of_variables = set(parsed_cnf)
    
    # check if negation exists
    for elem in non_negated_variables:
        if (elem+num_variables) in set_of_variables:
            vertices_with_negation.append(elem)
    
    pen_H = create_zero_hamiltonian(num_qubits)
    # Go through all vertices for which a negation exists in the CNF
    for vertex in vertices_with_negation:
        vertex_H = 1
        # Loop over all the qubits (since we want to apply operators in good order)
        for i in range(num_qubits):
            # find the qubit index for the vertex whose negation exists
            if vq_mapping[vertex] == i:
                vertex_H = vertex_H ^ vp_mapping[vertex]
            # find the qubit index for the vertex corresponding to the negation
            elif vq_mapping[vertex + num_variables] == i:
                vertex_H = vertex_H ^ vp_mapping[vertex + num_variables]
            # otherwise we just apply I
            else:
                vertex_H = vertex_H ^ I
        pen_H += vertex_H
    return pen_H
        

In [39]:
def create_total_hamiltonian(num_qubits, parsed_cnf, num_variables, vq_mapping, vp_mapping, M):
    prob_H = create_problem_hamiltonian(num_qubits, parsed_cnf, vq_mapping, vp_mapping)
    pen_H = create_penalty_hamiltonian(num_qubits, parsed_cnf, num_variables, vq_mapping, vp_mapping)
    return prob_H + M * pen_H

## Example Hamiltonian

In [16]:
#M_FACTOR = 10
#num_qubits = 4

# Create the Problem Hamiltonian

#H = (X ^ X ^ I ^ I) + (X ^ I ^ I ^ X) + (I ^ X ^ I ^ X) + (Y ^ I ^ Z ^ I)
#H += (Y ^ I ^ I ^ Z) + (I ^ I ^ Z ^ Z) + (Z ^ Y ^ I ^ I) + (Z ^ I ^ X ^ I)
#H += (I ^ Y ^ X ^ I) + (I ^ I ^ Y ^ Y) + (I ^ Z ^ Y ^ I) + (I ^ Z ^ I ^ Y)
#H += M_FACTOR * ((X ^ I ^ Z ^ I) + (Z ^ X ^ I ^ I) + (Y ^ I ^ I ^ X) + (I ^ Y ^ I ^ Y) + (I ^ Z ^ X ^ I) + (I ^ I ^ Y ^ Z))

## Recreate Eigenstate

Now that we found the optimal parameters for the circuit, we can recreate $\ket{\Psi(\theta^{(1)}_{opt}, ..., \theta^{(N)}_{opt})}$ using the circuit below.

In [40]:
def circuit_create_eigenstate(optimal_parameters, ansatz_arg, backend):
    # Apply the optimal parameters to the ansatz (which will give us a circuit for recreating |Psi>)
    ansatz_arg.assign_parameters(optimal_parameters, ansatz_arg.parameters)
    ansatz_opt = transpile(ansatz_arg, backend=backend)
    return ansatz_opt

## The Global State
We have to remember that, (for the PDF example SAT-NAE) after this circuit is executed, the global quantum state will be close to

$\begin{aligned}
\rho(\textbf{w}) = \frac{1}{2}(I + \frac{(-1)^{w_1}}{\sqrt{3}}X + \frac{(-1)^{w_3}}{\sqrt{3}}Y + \frac{(-1)^{w_8}}{\sqrt{3}}Z) \\
\otimes \frac{1}{2}(I + \frac{(-1)^{w_2}}{\sqrt{3}}X + \frac{(-1)^{w_4}}{\sqrt{3}}Y + \frac{(-1)^{w_{11}}}{\sqrt{3}}Z) \\
\otimes \frac{1}{2}(I + \frac{(-1)^{w_5}}{\sqrt{3}}X + \frac{(-1)^{w_6}}{\sqrt{3}}Y + \frac{(-1)^{w_7}}{\sqrt{3}}Z) \\
\otimes \frac{1}{2}(I + \frac{(-1)^{w_9}}{\sqrt{3}}X + \frac{(-1)^{w_{10}}}{\sqrt{3}}Y + \frac{(-1)^{w_{12}}}{\sqrt{3}}Z) \\
\end{aligned}$

Therefore, in order to get back the weights, we have to measure each qubit in three different bases ($X$, $Y$, and $Z$) in order to get back the bits $\{w_1, \dots, w_{12}\}$.

In total, we therefore need maximum 3 runs on the circuit (multiplied by the number of shots we want).

## Creating the three circuits
We want to create 3 circuits (one for each measurement basis). To measure in the Z basis, we can simply apply the measurement operator since qiskit already measures in the Z basis. For measuring in the X basis, we can apply an H (Hadamard) gate to each qubit, and for measuring in the Y basis, we can apply the S and then the H gate to each qubit.

In [41]:
def create_measurement_circuits(eigenstate_creator, num_qubits):
    # Circuit that will be measured in the X (and Y) basis simultaneously
    circuit_XY = QuantumCircuit(num_qubits, num_qubits)
    # Circuit that will be measured in the Z basis
    circuit_Z = QuantumCircuit(num_qubits, num_qubits)
    circuit_XY.reset(range(num_qubits))
    circuit_Z.reset(range(num_qubits))
    # Append to the circuit the circuit that creates |Psi>
    circuit_XY = circuit_XY.compose(eigenstate_creator)
    circuit_Z = circuit_Z.compose(eigenstate_creator)
    # Apply Hadamard gate to measure all the qubits in the X (or Y) basis
    circuit_XY.h(list(range(num_qubits)))
    circuit_XY.barrier(range(num_qubits))
    # Apply nothing if we want to measure in the Z basis
    circuit_Z.barrier(range(num_qubits))

    # Apply the measurement operators
    for i in range(num_qubits):
        for circuit in [circuit_XY, circuit_Z]:
            circuit.measure(qubit=i, cbit=i)
    return circuit_XY, circuit_Z

## Getting the results
Now let us run the circuits and fetch the results. But first we need to get an IBMQ backend.

In [42]:
def run_circuits(circuit_XY, circuit_Z, shots, backend):
    jobs = {}
    jobs["XY"] = backend.run(circuit_XY, shots=shots)
    jobs["Z"] = backend.run(circuit_Z, shots=shots)
    return jobs

## Get the results into the vertices

In [43]:
def get_results(jobs, qv_mapping):
    countsXY = jobs["XY"].result().get_counts()
    countsZ = jobs["Z"].result().get_counts()
    # Get the result with highest occurence, and reverse since qiskit returns results in little-endian 
    # Apply int() on them since qiskit gives us strings
    XY_results = list(map(lambda elem: int(elem), max(countsXY, key=countsXY.get)))[::-1]
    Z_results = list(map(lambda elem: int(elem), max(countsZ, key=countsZ.get)))[::-1]
    # the lists containing the results
    X_list = []
    Y_list = []
    Z_list = []
    for i in range(len(qv_mapping)): # list of vertices encoded into q[i]
        # go through the qubit->vertex mappings and append the vertex to the corresponding list
        for j in range(len(qv_mapping[i])):
            if j == 0:
                X_list.append(qv_mapping[i][j])
            elif j == 1:
                Y_list.append(qv_mapping[i][j])
            else:
                Z_list.append(qv_mapping[i][j])
                
    v_res_mapping = {}
    # Based on the results, get the results into a dictionary mapping vertex to assignment (0 or 1)
    for i in range(len(X_list)):
        v_res_mapping[X_list[i]] = XY_results[i]
    for i in range(len(Y_list)):
        v_res_mapping[Y_list[i]] = XY_results[i]
    for i in range(len(Z_list)):
        v_res_mapping[Z_list[i]] = Z_results[i]
        
    return v_res_mapping

In [44]:
# Calculate the error of our variable assignments (results)
def calculate_error(parsed_cnf, results, num_variables):
    satnae_error = 0
    consistency_error = 0
    
    # Calculate satnae error (when a clause is not satisfied or the NAE constraint does not hold)
    for i in range(0, len(parsed_cnf), 3):
        v1, v2, v3 = parsed_cnf[i], parsed_cnf[i+1], parsed_cnf[i+2]
        if results[v1] + results[v2] + results[v3] == 0 or results[v1] + results[v2] + results[v3] == 3:
            satnae_error += 1
            
    # Calculate consistency error (when a variable and its negation are assigned to the same value)
    for i in range(num_variables):        
        if results.get(i+num_variables) != None and results[i] == results[i+num_variables]:
            consistency_error += 1
    return satnae_error, consistency_error

In [45]:
def show_results(parsed_cnf, results, num_variables):
    print(f"For CNF: {parsed_cnf}")
    for vertex in results:
        if vertex < num_variables:
            if vertex + num_variables in results:
                print(f"x{vertex} = {results[vertex]} and not(x{vertex}) = {results[vertex+num_variables]}")
            else:
                print(f"x{vertex} = {results[vertex]}")

In [46]:
def calculate_satnae(M, parsed_cnf, obtained_graph, num_variables, show_ham=False, show_res=True):
    colors = color_graph(obtained_graph)
    nodes_grouped_by_color = group_nodes_by_color(obtained_graph, colors)
    qv_mapping = create_qv_mapping(nodes_grouped_by_color)
    num_qubits = len(qv_mapping)
    vq_mapping = create_vq_mapping(qv_mapping)
    vp_mapping = create_vp_mapping(qv_mapping)
    
    total_H = create_total_hamiltonian(num_qubits, parsed_cnf, num_variables, vq_mapping, vp_mapping, M)
    if show_ham:
        print(total_H)
    
    ansatz = TwoLocal(num_qubits, "ry", "cx", entanglement="linear", reps=1)
    
    ansatz_opt = transpile(ansatz, backend=backend)
    vqe = VQEClient(ansatz=ansatz_opt, optimizer=NFT(maxiter=96), provider=provider, backend=backend)
    result = vqe.compute_minimum_eigenvalue(total_H)
    best_parameters = result.optimal_point
    eigenstate_creator = circuit_create_eigenstate(best_parameters, ansatz_opt, backend)
    circuit_XY, circuit_Z = create_measurement_circuits(eigenstate_creator, num_qubits)
    jobs = run_circuits(circuit_XY, circuit_Z, 2048, backend)
    results = get_results(jobs, qv_mapping)
    satnae_error, consistency_error = calculate_error(parsed_cnf, results, num_variables)
    
    if show_res:
        show_results(parsed_cnf, results, num_variables)
    print("SATNAE_ERROR:", satnae_error, ", CONSISTENCY_ERROR:", consistency_error)
    return results, satnae_error, consistency_error

In [48]:
# Get the CNF from user input
obtained_graph, parsed_cnf, num_variables = parse_cnf_formula()
# example: 0 1 n2,2 n0 n5,n1 3 4,n3 n4 5

In [52]:
M = 20

In [54]:
total_sne = 0
total_ce = 0
total_succ = 0
num_tries = 1
for i in range(num_tries):
    print(f"Run {i+1}")
    print("------------")
    results, satnae_error, consistency_error = calculate_satnae(M, parsed_cnf, obtained_graph, num_variables, show_res=True, show_ham=False)
    total_sne += satnae_error
    total_ce += consistency_error
    if satnae_error + consistency_error == 0:
        total_succ += 1
print("average sat-nae error:", total_sne/num_tries)
print("average consistency error:", total_ce/num_tries)
print("total successes:", total_succ)

Run 1
------------
For CNF: [0, 1, 8, 2, 6, 11, 7, 3, 4, 9, 10, 5]
x5 = 1 and not(x5) = 1
x2 = 0 and not(x2) = 1
x1 = 0 and not(x1) = 0
x0 = 0 and not(x0) = 1
x4 = 0 and not(x4) = 1
x3 = 0 and not(x3) = 1
SATNAE_ERROR: 2 , CONSISTENCY_ERROR: 2
average sat-nae error: 2.0
average consistency error: 2.0
total successes: 0
